## Define model

In [1]:
from Model import *
model = get_embedding_cnn((80, 237, 1))

## Number of parameters

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 80, 228, 16)       176       
_________________________________________________________________
activation (Activation)      (None, 80, 228, 16)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 114, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 108, 64)       7232      
_________________________________________________________________
activation_1 (Activation)    (None, 80, 108, 64)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 80, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 80, 51, 64)        1

## FLOPs

In [3]:
from keras_flops import get_flops
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {flops / 10 ** 6:.06} M")

FLOPs: 498.337 M


## Memory usage

In [4]:
def keras_model_memory_usage_in_bytes(model, *, batch_size: int):
    """
    Source: https://stackoverflow.com/questions/43137288/how-to-determine-needed-memory-of-keras-model
    Return the estimated memory usage of a given Keras model in bytes.
    This includes the model weights and layers, but excludes the dataset.

    The model shapes are multipled by the batch size, but the weights are not.

    Args:
        model: A Keras model.
        batch_size: The batch size you intend to run the model with. If you
            have already specified the batch size in the model itself, then
            pass `1` as the argument here.
    Returns:
        An estimate of the Keras model's memory usage in bytes.

    """
    default_dtype = tf.keras.backend.floatx()
    shapes_mem_count = 0
    internal_model_mem_count = 0
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model):
            internal_model_mem_count += keras_model_memory_usage_in_bytes(
                layer, batch_size=batch_size
            )
        single_layer_mem = tf.as_dtype(layer.dtype or default_dtype).size
        out_shape = layer.output_shape
        if isinstance(out_shape, list):
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in model.trainable_weights]
    )
    non_trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in model.non_trainable_weights]
    )

    total_memory = (
        batch_size * shapes_mem_count
        + internal_model_mem_count
        + trainable_count
        + non_trainable_count
    )
    return total_memory

In [5]:
mem_bytes = keras_model_memory_usage_in_bytes(model=model, batch_size=1)
print(f"Memory usage: {mem_bytes / 10 ** 6:.03} MB")

Memory usage: 32.3 MB
